<a href="https://colab.research.google.com/github/Bombbom/ML4SecurityProject/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Thu thập dataset


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install hexbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 32.6 MB/s 
     |████████████████████████████████| 212 kB 67.8 MB/s 
     |████████████████████████████████| 182 kB 59.7 MB/s 
     |████████████████████████████████| 132 kB 60.6 MB/s 
     |████████████████████████████████| 127 kB 67.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import pandas as pd
import seaborn as sns
from hexbytes import HexBytes
import matplotlib.pyplot as plt
from datasets import load_dataset

In [ ]:
train_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='train', ignore_verifications=True)
# test_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='test', ignore_verifications=True)
# val_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='validation', ignore_verifications=True)

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/79641 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10861 [00:00<?, ? examples/s]

Dataset slither-audited-smart-contracts downloaded and prepared to /root/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13. Subsequent calls will reuse this data.


## Dataset info

In [ ]:
train_set.shape

(79641, 4)

In [ ]:
train_set

Dataset({
    features: ['address', 'source_code', 'bytecode', 'slither'],
    num_rows: 79641
})

In [ ]:
data = pd.DataFrame(train_set)

- 0:'access-control', 1:'arithmetic', 2:'other', 3:'reentrancy', 4:'safe', 5:'unchecked-calls'

In [ ]:
data

,address,source_code,bytecode,slither
0,0x006699d34AA3013605d468d2755A2Fe59A16B12B,pragma solidity 0.5.4;\n\ninterface IERC20 {\n...,0x608060405234801561001057600080fd5b5060043610...,[1]
1,0x00c83aeCC790e8a4453e5dD3B0B4b3680501a7A7,pragma solidity 0.6.10;\npragma experimental A...,0x608060405234801561001057600080fd5b5060043610...,"[1, 2, 3]"
2,0x010c5322d78c88ca18282b0a072a8913648b3038,pragma solidity 0.6.5;\npragma experimental AB...,0x608060405234801561001057600080fd5b5060043610...,[4]
3,0x0114622386c1a00686e594c70682d7aa0f8afa29,pragma solidity 0.6.6;\n\nlibrary Address {\n\...,0x608060405234801561001057600080fd5b5060043610...,[4]
4,0x01b952402442630232f32d48380a557015b7c5ec,pragma solidity 0.7.4;\n\nenum MathError { NO_...,0x608060405234801561001057600080fd5b5060043610...,"[5, 2]"
...,...,...,...,...
79636,0x80d19b2bf1743371855732971cec24fe9dcd1417,/**\n *Submitted for verification at Etherscan...,0x7380d19b2bf1743371855732971cec24fe9dcd141730...,[4]
79637,0x810b54cdb3d8c97d857b500cdfdb5715e1d27837,contract ERC20Basic {\n uint256 public totalS...,0x6060604052600436106100c5576000357c0100000000...,[4]
79638,0x8142871ce8e9b69af8b50369fde8f9959d699fa6,contract Bodies2 {\n\nstring public constant b...,0x608060405234801561001057600080fd5b5060043610...,[4]
79639,0x8154094af8dc315b31c6ec746260003ee828fd93,contract LifeSet_005 {\n \n// part.I_fixe.....,0x6080604052600436106100da5763ffffffff7c010000...,[4]


- function to remove comments in source code smart contract

In [ ]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group is not None, then we have captured a real comment string.
        if match.group(2) is not None:
            return "" 
        else: # otherwise, we will return the 1st group
            return match.group(1) 
    return regex.sub(_replacer, string)

In [ ]:
column = ["address", "source_code","bytecode",	"slither","access-control", "arithmetic", "other", "reentrancy", "safe", "unchecked-calls"]
df = pd.DataFrame(columns = column)

In [ ]:
data.loc[0]

address               0x006699d34AA3013605d468d2755A2Fe59A16B12B
source_code    pragma solidity 0.5.4;\n\ninterface IERC20 {\n...
bytecode       0x608060405234801561001057600080fd5b5060043610...
slither                                                      [1]
Name: 0, dtype: object

In [ ]:
for i in range(len(data)):
  row = list(data.loc[i])
  # print(list(row))
  label = row[3]
  row[1] = remove_comments(row[1])
  vul = [0,0,0,0,0,0]
  for i in label:
    vul[i]=1
  row.extend(vul)
  length = len(df)
  df.loc[length] = row

In [ ]:
df.to_csv('/content/drive/MyDrive/Submit/NT522_Project/train_set_big_multilabel_exploid.csv',index=False)

In [ ]:
df

,address,source_code,bytecode,slither,access-control,arithmetic,other,reentrancy,safe,unchecked-calls
0,0x006699d34AA3013605d468d2755A2Fe59A16B12B,pragma solidity 0.5.4;\n\ninterface IERC20 {\n...,0x608060405234801561001057600080fd5b5060043610...,[1],0,1,0,0,0,0
1,0x00c83aeCC790e8a4453e5dD3B0B4b3680501a7A7,pragma solidity 0.6.10;\npragma experimental A...,0x608060405234801561001057600080fd5b5060043610...,"[1, 2, 3]",0,1,1,1,0,0
2,0x010c5322d78c88ca18282b0a072a8913648b3038,pragma solidity 0.6.5;\npragma experimental AB...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
3,0x0114622386c1a00686e594c70682d7aa0f8afa29,pragma solidity 0.6.6;\n\nlibrary Address {\n\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
4,0x01b952402442630232f32d48380a557015b7c5ec,pragma solidity 0.7.4;\n\nenum MathError { NO_...,0x608060405234801561001057600080fd5b5060043610...,"[5, 2]",0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
79636,0x80d19b2bf1743371855732971cec24fe9dcd1417,\n\nlibrary IterableMapping {\n \n struc...,0x7380d19b2bf1743371855732971cec24fe9dcd141730...,[4],0,0,0,0,1,0
79637,0x810b54cdb3d8c97d857b500cdfdb5715e1d27837,contract ERC20Basic {\n uint256 public totalS...,0x6060604052600436106100c5576000357c0100000000...,[4],0,0,0,0,1,0
79638,0x8142871ce8e9b69af8b50369fde8f9959d699fa6,contract Bodies2 {\n\nstring public constant b...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
79639,0x8154094af8dc315b31c6ec746260003ee828fd93,contract LifeSet_005 {\n \n\n\n uint256 pu...,0x6080604052600436106100da5763ffffffff7c010000...,[4],0,0,0,0,1,0


- Read - 10000 record

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Submit/NT522_Project/train_set_big_multilabel_exploid.csv', index_col=None)

## Balance dataset

In [3]:
data_safe=df[df['safe']==1][:5000]

In [4]:
data_safe.shape

(5000, 10)

In [5]:
data_reentrancy = df[df['reentrancy']==1][:5000]

In [6]:
data_reentrancy.shape

(5000, 10)

In [7]:
new_data = pd.concat([data_safe,data_reentrancy]).reset_index(drop=True)

In [8]:
new_data.shape

(10000, 10)

In [9]:
new_data.head()

,address,source_code,bytecode,slither,access-control,arithmetic,other,reentrancy,safe,unchecked-calls
0,0x010c5322d78c88ca18282b0a072a8913648b3038,pragma solidity 0.6.5;\npragma experimental AB...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
1,0x0114622386c1a00686e594c70682d7aa0f8afa29,pragma solidity 0.6.6;\n\nlibrary Address {\n\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
2,0x0258f474786ddfd37abce6df6bbb1dd5dfc4434a,pragma solidity 0.5.10;\n\ncontract Context {\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
3,0x0263e1dcfcaaad8fd4e840fe4b22854485a561f0,pragma solidity 0.6.6;\n\nlibrary Address {\n\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
4,0x0319a976c8dea5d31d1cf36eaffd8bec14957179,pragma solidity 0.6.5;\npragma experimental AB...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0


In [10]:
new_data

,address,source_code,bytecode,slither,access-control,arithmetic,other,reentrancy,safe,unchecked-calls
0,0x010c5322d78c88ca18282b0a072a8913648b3038,pragma solidity 0.6.5;\npragma experimental AB...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
1,0x0114622386c1a00686e594c70682d7aa0f8afa29,pragma solidity 0.6.6;\n\nlibrary Address {\n\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
2,0x0258f474786ddfd37abce6df6bbb1dd5dfc4434a,pragma solidity 0.5.10;\n\ncontract Context {\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
3,0x0263e1dcfcaaad8fd4e840fe4b22854485a561f0,pragma solidity 0.6.6;\n\nlibrary Address {\n\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
4,0x0319a976c8dea5d31d1cf36eaffd8bec14957179,pragma solidity 0.6.5;\npragma experimental AB...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
9995,0x4237f6D8725757599003a699d296C19890185205,\n\n\n\npragma solidity 0.6.12;\n\n\n\n\n\nabs...,0x6080604052600436106103dd5760003560e01c80637d...,"[1, 5, 2, 3]",0,1,1,1,0,1
9996,0x42389ae36d2406dd3c1f45d024341456b62e5c50,\n\n\n\npragma solidity ^0.8.4;\n\nabstract co...,0x60806040526004361061012e5760003560e01c80638d...,"[5, 3]",0,0,0,1,0,1
9997,0x423d0b4f9a99552ffb1d5eaee48324d5d40f853e,\n\n\n\n\npragma solidity ^0.8.0;\n\n\nlibrary...,0x608060405260043610620003f75760003560e01c8063...,"[5, 3]",0,0,0,1,0,1
9998,0x423d71d1b4537bd885fe74667325e2fea141e5ee,\n\n\npragma solidity 0.8.9;\n \nabstract cont...,0x6080604052600436106103855760003560e01c806392...,"[1, 0, 5, 3]",1,1,0,1,0,1


In [11]:
new_data = new_data.sample(frac=1, replace=False ,random_state=12)

In [12]:
new_data.reset_index(drop=True)

,address,source_code,bytecode,slither,access-control,arithmetic,other,reentrancy,safe,unchecked-calls
0,0x07ab4886566b7ab782a9d627e8554db66fb06357,\n\n\n\npragma solidity ^0.8.0;\n\n\nabstract ...,0x608060405234801561001057600080fd5b5060043610...,"[2, 1, 3]",0,1,1,1,0,0
1,0x3207f9132c0b9f39fa01c028564851178c59a3f3,\n\n\n\npragma solidity ^0.5.0;\n\n\nlibrary M...,0x608060405234801561001057600080fd5b5060043610...,"[1, 3]",0,1,0,1,0,0
2,0x232391765aacf9e075e66162ce4e453afcbf14b5,\n\n\n\npragma solidity 0.8.2;\n\n\nabstract c...,0x6080604052600436106100e15760003560e01c806372...,[4],0,0,0,0,1,0
3,0x3109d8eec2cc8727b852aeb702d5c05c6138b66e,pragma solidity ^0.4.20;\ncontract Ownable {\n...,0x60806040526004361061013a5763ffffffff60e06002...,"[5, 2, 3]",0,0,1,1,0,1
4,0x11b1eb6f3cb0f50d9a3cf0fbc2d83e7ebc956e9d,\n\n\n\n\n\npragma solidity ^0.8.4;\n\nabstrac...,0x6080604052600436106101025760003560e01c806370...,"[5, 1, 3]",0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
9995,0x293a191197A766f588d21B8F29fC059984246c64,\n\n\n\n\n\n\n\n\n\n\npragma solidity ^0.8.0;\...,0x608060405234801561001057600080fd5b5060043610...,[4],0,0,0,0,1,0
9996,0x1f794cd47d2c79aee4d6358825957979665be6c9,\n\n\npragma solidity ^0.8.4;\n\nabstract cont...,0x60806040526004361061016a5760003560e01c806370...,"[0, 5, 3]",1,0,0,1,0,1
9997,0x24942356F818E73b6937f5D6366105C6A208a23D,\npragma solidity ^0.4.11;\n\n\n\ncontract Own...,0x606060405236156100a15763ffffffff7c0100000000...,[4],0,0,0,0,1,0
9998,0x3c9ded5b4b42b3e308ee46dcfe3819104772aad3,\n\n\n\n\n\n\n\n\n\npragma solidity ^0.8.13;\n...,0x60806040526004361061012e5760003560e01c806371...,"[0, 5, 3]",1,0,0,1,0,1


In [13]:
new_data.to_csv('/content/drive/MyDrive/Submit/NT522_Project/random_5000_reentrancy_5000_safe.csv',index=False)

## Create dataset.txt

In [ ]:
row=list(new_data.iloc[0])

In [ ]:
row

['0x9f270816aeb7add2cdbec92bbdd350edbdeaf5f7',
 'pragma solidity ^0.5.0;\n\n\ncontract ERC20Interface {\n    function totalSupply() public view returns (uint);\n    function balanceOf(address tokenOwner) public view returns (uint balance);\n    function allowance(address tokenOwner, address spender) public view returns (uint remaining);\n    function transfer(address to, uint tokens) public returns (bool success);\n    function approve(address spender, uint tokens) public returns (bool success);\n    function transferFrom(address from, address to, uint tokens) public returns (bool success);\n\n    event Transfer(address indexed from, address indexed to, uint tokens);\n    event Approval(address indexed tokenOwner, address indexed spender, uint tokens);\n}\n\n\n\n\ncontract SafeMath {\n    function safeAdd(uint a, uint b) public pure returns (uint c) {\n        c = a + b;\n        require(c >= a);\n    }\n    function safeSub(uint a, uint b) public pure returns (uint c) {\n        requi

In [ ]:
file = row[1].split('\n')

In [ ]:
file

['pragma solidity ^0.5.0;',
 '',
 '',
 'contract ERC20Interface {',
 '    function totalSupply() public view returns (uint);',
 '    function balanceOf(address tokenOwner) public view returns (uint balance);',
 '    function allowance(address tokenOwner, address spender) public view returns (uint remaining);',
 '    function transfer(address to, uint tokens) public returns (bool success);',
 '    function approve(address spender, uint tokens) public returns (bool success);',
 '    function transferFrom(address from, address to, uint tokens) public returns (bool success);',
 '',
 '    event Transfer(address indexed from, address indexed to, uint tokens);',
 '    event Approval(address indexed tokenOwner, address indexed spender, uint tokens);',
 '}',
 '',
 '',
 '',
 '',
 'contract SafeMath {',
 '    function safeAdd(uint a, uint b) public pure returns (uint c) {',
 '        c = a + b;',
 '        require(c >= a);',
 '    }',
 '    function safeSub(uint a, uint b) public pure returns (ui

In [ ]:
for line in file:
  line = line.strip()
  if len(line)>0:
    print(line)

pragma solidity ^0.5.0;
contract ERC20Interface {
function totalSupply() public view returns (uint);
function balanceOf(address tokenOwner) public view returns (uint balance);
function allowance(address tokenOwner, address spender) public view returns (uint remaining);
function transfer(address to, uint tokens) public returns (bool success);
function approve(address spender, uint tokens) public returns (bool success);
function transferFrom(address from, address to, uint tokens) public returns (bool success);
event Transfer(address indexed from, address indexed to, uint tokens);
event Approval(address indexed tokenOwner, address indexed spender, uint tokens);
}
contract SafeMath {
function safeAdd(uint a, uint b) public pure returns (uint c) {
c = a + b;
require(c >= a);
}
function safeSub(uint a, uint b) public pure returns (uint c) {
require(b <= a); c = a - b; } function safeMul(uint a, uint b) public pure returns (uint c) { c = a * b; require(a == 0 || c / a == b); } function safeDi

In [ ]:
del str

In [14]:
count = 0
f_w = open("/content/drive/MyDrive/Submit/NT522_Project/dataset_10000_record.txt", "a")

for i in range(len(new_data)):
    row = list(new_data.loc[i])
    name = row[0] + ".sol"
    label = row[7] #reentrancy 
    count +=1
    f_w.write(str(count) + " " + str(name) + "\n")
    f_w.write(str(name)+"\n")
    codes = []
    f = row[1].split('\n')
    
    for line in f:
        text = line.strip()
        if text!= "":
            codes.append(text)
    
    for k in range(0,len(codes)):
        f_w.write(codes[k] + "\n")
    
    f_w.write(str(label)+"\n")
    f_w.write("---------------------------------" + "\n")

## Dataset (code snippet)